In [2]:
import gspread
import time 
from datetime import datetime
import re
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import pandas as pd


In [6]:
class ScrapearGMaps:

    data = {}
    worksheet = {}


    def __init__(self) -> None:
        # Iniciamos el webdriver
        self.driver = webdriver.Chrome()

    # Funcion para scrollear
    def scroll_page (self):
        # Identificamos el objeto hacia el que queremos hacer el scroll
        section_loading = self.driver.find_element(By.CLASS_NAME, 'TFQHme ')

        # Hacer scroll en los resultados
        for _ in range(30):  # Ajustar el rango según la cantidad de scrolls que necesites
            scrollable_div = self.driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]')
            self.driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
            time.sleep(3)  # Esperar un poco antes de hacer el siguiente scroll

    # Funcion para obtener el nombre
    def get_name (self):
        try:
            # Por medio del Xpath extraemos el texto del nombre
            return self.driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[1]/h1').text
        except:
            return "SD"
        
    def get_coord (self,link):
        coordenadas = re.search(r"!3d-?\d\d?\.\d{4,8}!4d-?\d\d?\.\d{4,8}", link).group()
        coordenada = coordenadas.replace('!3d','')

        return tuple(coordenada.split('!4d'))
    
    def get_gmapid(self,link):
        patron = r'0x[0-9a-fA-F]+:0x[0-9a-fA-F]+'

        try:
            match = re.search(patron, link)

            extracted_data = match.group()
            return extracted_data

        except:
            return 'Extraccion fallida'
        

    def get_category (self):

        categoria=self.driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[2]/div/div[2]/span[1]/span/button').text

        return categoria
    
    def get_rating (self):
        rating = self.driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[2]/div/div[1]/div[2]/span[1]/span[1]').text
        rating = rating.replace(',','.')
        
        return float(rating)
    
    def get_reviews (self):

        reviews = self.driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[2]/div/div[1]/div[2]/span[2]/span/span').text
        reviews = reviews.replace('.','')
        reviews = reviews.replace('(','')
        reviews = reviews.replace(')','')
        
        return int(reviews)
    
    def get_direccion (self):
            try:
                direccion = self.driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[9]/div[3]/button/div/div[2]/div[1]').text

                return direccion
            except:
                direccion = self.driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[11]/div[3]/button/div/div[2]/div[1]').text

                return direccion

    # Creamos una funcion para extraer la ciudad desde la direccion
    def extraer_ciudad(self, direccion):

        try:
            # Dividir la dirección en partes usando la coma como separador
            partes = direccion.split(", ")

            # Asegurarse de que haya al menos tres partes (ciudad, estado_código postal)
            if len(partes) >= 3:
                ciudad = partes[1]  # Extraer la 3ra parte como ciudad
                return ciudad
            else:
                return None # Devolver None para ciudad y estado si tiene la configuración planteada
        except IndexError:
            return None  # Devolver None para ciudad y estado si la extracción falla
    
    def get_reviews_completas (self):

        self.driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[3]/div/div/button[2]/div[2]/div[2]')


    def scraping (self,link):

        name = []
        gmap_id = []
        latitud = []
        longitud = []
        categoria = []
        avg_rating = []
        reviews = []
        ciudad = []


        self.driver.get(link)

        time.sleep(5)

        self.scroll_page()

        lugares = self.driver.find_elements(By.CLASS_NAME, 'hfpxzc')
        cantidad = len(lugares)

        for i in range(0,cantidad):
            
            try:

                lugar = lugares[i]

                scrollable_div = self.driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]')
                self.driver.execute_script("arguments[0].scrollIntoView();", lugar)

                time.sleep(2)
                lugar.click()

                time.sleep(4)

                name_temp = self.get_name()
                mapid_temp = self.get_gmapid(self.driver.current_url)
                coordenadas = self.get_coord(self.driver.current_url)
                categoria_temp = self.get_category()
                rating_temp = self.get_rating()
                reviews_temp = self.get_reviews()
                direccion = self.get_direccion()
                ciudad_temp = self.extraer_ciudad(direccion)


                name.append(name_temp)
                gmap_id.append(mapid_temp)
                latitud.append(coordenadas[0])
                longitud.append(coordenadas[1])
                categoria.append(categoria_temp)
                avg_rating.append(rating_temp)
                reviews.append(reviews_temp)
                ciudad.append(ciudad_temp)
                time.sleep(2)
            except:

                name.append(None)
                gmap_id.append(None)
                latitud.append(None)
                longitud.append(None)
                categoria.append(None)
                avg_rating.append(None)
                reviews.append(None)
                ciudad.append(None)
                time.sleep(2)


        df = pd.DataFrame({'name':name,
                            'gmap_id':gmap_id,
                            'latitude':latitud,
                            'longitude':longitud,
                            'category':categoria,
                            'avg_rating':avg_rating,
                            'num_of_reviws':reviews,
                            'city':ciudad})
            
        return df

In [7]:
link = 'https://www.google.com/maps/search/Restaurantes,+Florida,+EEUU/@28.4613115,-91.2139021,6z/data=!4m2!2m1!6e5?entry=ttu'

gscraping = ScrapearGMaps()

df = gscraping.scraping(link)

In [8]:
df

,name,gmap_id,latitude,longitude,category,avg_rating,num_of_reviws,city
0,El Cilantrillo Restaurant,0x88e77dddfa7e805d:0xf115633fcf003d5,28.4501349,-81.3969286,Restaurante puertorriqueño,4.6,3725,Orlando
1,Rusty Pelican Miami,0x88d9b5ce30dc6373:0xb44eee30744447fe,25.7478726,-80.1756055,Marisquería,4.7,17313,Key Biscayne
2,CVI.CHE 105,0x88d9b69e13f63e97:0xf33305312982752d,25.7754765,-80.1884418,Restaurante peruano,4.8,20366,Miami
3,Bolivar Restaurant Bar,0x88d9b48d331b8829:0x5596e06af04e632f,25.779079,-80.1329652,Restaurante colombiano,4.5,2321,Miami Beach
4,Crazy About You,0x88d9b681bd4b567b:0x4eef938c35d01d59,25.7620008,-80.1889886,Restaurante americano,4.7,11348,Miami
...,...,...,...,...,...,...,...,...
115,Casita Tejas Mexican Restaurant,0x88d9e740086e63af:0x640fb304b68f804c,25.4704744,-80.4771906,Restaurante tex-mex,4.3,1606,Homestead
116,BACÁN,0x88dd89c0e8c9326b:0x64b95017c8e50fbc,28.3738554,-81.2815686,Restaurante latinoamericano,4.6,549,Orlando
117,Burger King,0x88de69862930a12d:0xb89855564d571f57,27.840249,-80.487037,Restaurante,3.1,372,Sebastian
118,Ponderosa Steakhouse,0x88dd7ebaf9deb90f:0x9ee7bac84e298a1c,28.3333706,-81.5899512,Restaurante especializado en filetes,3.7,4299,Kissimmee


In [31]:
df.to_parquet('../../Data/Parquet/Scraping_sitios_google.parquet', index=False)

In [3]:
import time
from selenium import webdriver

driver = webdriver.Chrome()
driver.get('https://www.google.com/maps/search/Restaurantes,+Florida,+EEUU/@28.4613115,-91.2139021,6z/data=!4m2!2m1!6e5?entry=ttu')
time.sleep(5)

section_loading = driver.find_element(By.CLASS_NAME, 'TFQHme ')

# Hacer scroll en los resultados
for _ in range(30):  # Ajustar el rango según la cantidad de scrolls que necesites
    scrollable_div = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]')
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
    time.sleep(2)  # Esperar un poco antes de hacer el siguiente scroll


lugares = driver.find_elements(By.CLASS_NAME, 'hfpxzc')
lugar = lugares[0]

time.sleep(2)
lugar.click()
time.sleep (2)
nombre=driver.find_element(By.CLASS_NAME, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[1]/h1').text
print(nombre)

url = driver.current_url

print(url)

InvalidSelectorException: Message: invalid selector
from javascript error: {"status":32,"value":"An invalid or illegal selector was specified"}
  (Session info: chrome=126.0.6478.115); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalid-selector-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6731D3E32+31618]
	(No symbol) [0x00007FF67314B099]
	(No symbol) [0x00007FF67300888A]
	(No symbol) [0x00007FF67300E6BE]
	(No symbol) [0x00007FF673011011]
	(No symbol) [0x00007FF6730110B0]
	(No symbol) [0x00007FF67305805C]
	(No symbol) [0x00007FF67305862C]
	(No symbol) [0x00007FF67309F787]
	(No symbol) [0x00007FF67307D14F]
	(No symbol) [0x00007FF67309CA80]
	(No symbol) [0x00007FF67307CEB3]
	(No symbol) [0x00007FF67304A46B]
	(No symbol) [0x00007FF67304B001]
	GetHandleVerifier [0x00007FF6734D9FFD+3202381]
	GetHandleVerifier [0x00007FF673526A1D+3516269]
	GetHandleVerifier [0x00007FF67351C490+3473888]
	GetHandleVerifier [0x000

In [4]:
opiniones = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[3]/div/div/button[2]/div[2]/div[2]')
opiniones.click()
scrollable_div = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]')
driver.execute_script("arguments[0].scrollIntoView();", lugar)
ordenar = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[3]/div[7]/div[2]/button/span/span')

In [59]:
nombre=driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[1]/h1').text

print(nombre)

El Cilantrillo Restaurant


In [149]:
nombre

'Estefan Kitchen Orlando'

In [11]:
url = driver.current_url

In [13]:
print(url)

https://www.google.com/maps/place/El+Cilantrillo+Restaurant/@28.4613115,-91.2139021,6z/data=!4m11!1m3!2m2!1sRestaurantes,+Florida,+EEUU!6e5!3m6!1s0x88e77dddfa7e805d:0xf115633fcf003d5!8m2!3d28.4501349!4d-81.3969286!15sChtSZXN0YXVyYW50ZXMsIEZsb3JpZGEsIEVFVVVaGyIZcmVzdGF1cmFudGVzIGZsb3JpZGEgZWV1dZIBF3B1ZXJ0b19yaWNhbl9yZXN0YXVyYW50mgEkQ2hkRFNVaE5NRzluUzBWSlEwRm5TVVJoY0V0TWNHcFJSUkFC4AEA!16s%2Fg%2F11mvn2dhtn?entry=ttu


In [151]:
coordenadas = re.search(r"!3d-?\d\d?\.\d{4,8}!4d-?\d\d?\.\d{4,8}", url).group()
coordenada = coordenadas.replace('!3d','')

final = tuple(coordenada.split('!4d'))

final

('28.3455145', '-81.6086358')

In [152]:

pattern = r'0x[0-9a-fA-F]+:0x[0-9a-fA-F]+'

match = re.search(pattern, url)

if match:
    extracted_data = match.group()
    print("Dato extraído:", extracted_data)
else:
    print("No se encontró el patrón.")
# 0x88e77dddfa7e805d:0xf115633fcf003d5

Dato extraído: 0x88dd7dc69b3a06cb:0xfa20cb8563a46242


In [153]:

categoria=driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[2]/div/div[2]/span[1]/span/button').text

print(categoria)

Restaurante


In [154]:
rating = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[2]/div/div[1]/div[2]/span[1]/span[1]').text
rating = rating.replace(',','.')
float(rating)

4.1

In [155]:
reviews = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[2]/div/div[1]/div[2]/span[2]/span/span').text
reviews = reviews.replace('.','')
reviews = reviews.replace('(','')
reviews = reviews.replace(')','')
int(reviews)

1385

In [157]:
direccion = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[9]/div[3]/button/div/div[2]/div[1]').text
# //*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[9]/div[3]/button/div/div[2]/div[1]
# //*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[11]/div[3]/button/div/div[2]/div[1]
direccion

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[9]/div[3]/button/div/div[2]/div[1]"}
  (Session info: chrome=126.0.6478.63); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF633833E32+31618]
	(No symbol) [0x00007FF6337AB099]
	(No symbol) [0x00007FF63366888A]
	(No symbol) [0x00007FF6336B8524]
	(No symbol) [0x00007FF6336B862C]
	(No symbol) [0x00007FF6336FF787]
	(No symbol) [0x00007FF6336DD14F]
	(No symbol) [0x00007FF6336FCA80]
	(No symbol) [0x00007FF6336DCEB3]
	(No symbol) [0x00007FF6336AA46B]
	(No symbol) [0x00007FF6336AB001]
	GetHandleVerifier [0x00007FF633B39FFD+3202381]
	GetHandleVerifier [0x00007FF633B86A1D+3516269]
	GetHandleVerifier [0x00007FF633B7C490+3473888]
	GetHandleVerifier [0x00007FF6338E5D36+760454]
	(No symbol) 

In [99]:
lista = [1,2,3]
lista2 = [4,5,6]
len(lista)

3

In [103]:
import pandas as pd

df = pd.DataFrame({'a':lista, 'b':lista2})

df

,a,b
0,1,4
1,2,5
2,3,6


In [66]:
lugar = lugares[21]

time.sleep(2)
lugar.click()

In [65]:
scrollable_div = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]')
driver.execute_script("arguments[0].scrollIntoView();", lugar)